1. The first character is not a digit (other characters can be digits).
2. The first character is uppercase, all the other characters that are letters are lowercase.
3. There are less than 0.5% occurrences of this word, when this word regardless to its case appears in the dataset and the condition (2) is not met.

Order by quantity, most popular first, output format:

name <tab> count

The result is the 5th line in the output, for example:

john 1234

In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode


def isDigitStart(word):
    # 1. The first character is not a digit (other characters can be digits)
    return re.match("^[0-9]", word) is not None


def isName(word):
    # 2. The first character is uppercase , All the other characters that are letters are lowercase.
    return re.match(r"^[A-Z][^A-Z]*$", word) is not None


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
#         print >> sys.stderr, "reporter:counter:myGroup,Total_words,%d" % 1
        
        if isDigitStart(word):
            continue

        if isName(word):
            label = 'ok'
        else:
            label = 'not_ok'
            
        print "%s\t%d\t%s" % (word.lower(), 1, label)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum_ok = 0
word_sum_not_ok = 0

for line in sys.stdin:
    try:
        key, count, label = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    
    # New key
    if current_key != key:
        if current_key and float(word_sum_not_ok) / (word_sum_ok + word_sum_not_ok) <= 0.005:
            print "%s\t%d" % (current_key, word_sum_ok)
        word_sum_ok = 0
        word_sum_not_ok = 0
        current_key = key
    
    # Same key
    if label == 'ok':
        word_sum_ok += count
    else:
        word_sum_not_ok += count

if current_key and float(word_sum_not_ok) / (word_sum_ok + word_sum_not_ok) <= 0.005:
    print "%s\t%d" % (current_key, word_sum_ok)

Overwriting reducer.py


In [3]:
%%bash

OUT_DIR="wordcount_result"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

# Print result
hadoop fs -cat ${OUT_DIR}/part-* | sort -k2,2nr | head -5 | tail -1

# print log to stderr for grader
cat output.log >&2

french	5742


sort: write failed: 'standard output': Broken pipe
sort: write error
18/01/25 08:02:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/25 08:02:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/25 08:02:03 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/25 08:02:03 INFO mapreduce.JobSubmitter: number of splits:2
18/01/25 08:02:03 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516837645430_0012
18/01/25 08:02:04 INFO impl.YarnClientImpl: Submitted application application_1516837645430_0012
18/01/25 08:02:04 INFO mapreduce.Job: The url to track the job: http://c239dcf23262:8088/proxy/application_1516837645430_0012/
18/01/25 08:02:04 INFO mapreduce.Job: Running job: job_1516837645430_0012
18/01/25 08:02:10 INFO mapreduce.Job: Job job_1516837645430_0012 running in uber mode : false
18/01/25 08:02:10 INFO mapreduce.Job:  map 0% reduce 0%
18/01/25 08:02:26 INFO mapreduce.Job:  map 31% reduce 0%
18/01/25 08

## Test

In [4]:
# %%writefile debug_data.txt
# 1	5ffgfg truong Minh Xua.n ThAo 5 T4an 
# 2	Phuong PHuong Phuong PhUong PhuonG Ph9uong Phuong Phuong Phuong Ph9uong Phuong Phuong Phuong Phuong Phuong
# 3	Phuong Phuong Phuong Phuong PhuonG Phuong Phuong Phuong Phuong Phuong Phuong Phuong Phuong Phuong Phuong
# 4	Thao Thao Thao Thao Thao ThaO Thao Thao Thao Thao Thao Thao Thao

In [5]:
# cat debug_data.txt | python2 mapper.py | sort | python2 reducer.py